In [76]:
from html.parser import HTMLParser
import re
from urllib.parse import quote_plus
from nbformat import read
from nbconvert import MarkdownExporter
from nbconvert.writers import FilesWriter
from nbconvert.nbconvertapp import NbConvertApp
from nbconvert.preprocessors import Preprocessor
from bs4 import BeautifulSoup

In [67]:
notebook = './1-MonteCarlo-Resource-Assessments-part1.ipynb'

In [68]:
with open(notebook, 'r', encoding='utf-8') as f:
            nb = read(f, 4)
        
try:
    output = quote_plus(
        nb.metadata.date + '-' + nb.metadata.title, safe='/'
    )
except:
    output = ''

In [69]:
# class MyHTMLParser(HTMLParser):
    
#     def handle_starttag(self, tag, attrs):
#         if tag == 'img':
#             for a in attrs:
#                 if a[0] == 'src':
#                     files.append(a[1])


In [143]:
class CustomPreprocess(Preprocessor):

    
    def preprocess(self, nb, resources):
        
        global files
        files = []

        pre = """<div class="equation">\n\t<div>"""
        post = """</div>\n<div class="equation_dots"></div>\n"""
        post += "<div></div>\n"
        post += "</div>\n"
        
#         html_parser = MyHTMLParser()
        
        for cell in nb.cells:
            if cell.cell_type == 'markdown':
                s = re.sub("^\# .+\n\n", '', cell.source, re.MULTILINE)
                s = re.sub(r'\\\\\[', pre + r'\\[', s)
                s = re.sub(r'\\\\\]', r'\\]' + post, s)
                
                
                soup = BeautifulSoup(s, 'html.parser')
                for img in soup.findAll('img'):
                    print(img['src'])
                    files.append(img['src'])
                    img['src'] = '{{ site.url }}' + output + img['src']
                s = str(soup)
                
                
                cell.source = s
                
#                 html_parser.feed(s)
                
#                 files.append(1)
                
                
        try:
            nb.metadata.notebook
        except:
            nb.metadata.notebook = output
            
        return nb, resources

In [144]:
md_exporter = MarkdownExporter(
    template_name="blog-markdown",
    preprocessors=[CustomPreprocess]
)

In [145]:
md_writer = FilesWriter(
    build_directory = "/Users/dom/Downloads/nbconvert_output",
)


In [146]:
app = NbConvertApp(
    output_base = output
)
app.exporter = md_exporter
app.writer = md_writer

In [147]:
md_writer.files = files
app.convert_single_notebook(notebook)

[NbConvertApp] Converting notebook ./1-MonteCarlo-Resource-Assessments-part1.ipynb to 


./assets/1-sensitivity.png


In [148]:
files

['./assets/1-sensitivity.png']